I'll use this notebook to study potential LJ constituents and look into whether I can unambiguously pick out the relevant particles with dR requirements. This study should be updated to instead use pfjet.pfcand and compare directly with ljsources.

In [1]:
# python
import sys
import os
import importlib
# columnar analysis
from coffea import processor
import awkward as ak
#from dask.distributed import Client
# local
sys.path.insert(1, os.path.join(sys.path[0], '../..')) # fixme: there must be a better way to handle this...
from sidm.tools import ffschema, sidm_processor, utilities
# always reload local modules to pick up changes during development
importlib.reload(ffschema)
importlib.reload(sidm_processor)
importlib.reload(utilities)
# plotting
import matplotlib.pyplot as plt
utilities.set_plot_style()

Start by looking at default 2mu2e sample with 2mu2e selection and matched dark photons.

In [2]:
samples = [
    "2Mu2E_100GeV_1p2GeV_9p6mm",
]
fileset = utilities.make_fileset(samples, "ffntuple_v4")

In [3]:
runner = processor.Runner(
    #executor=processor.IterativeExecutor(),
    executor=processor.FuturesExecutor(),
    schema=ffschema.FFSchema,
)

p = sidm_processor.SidmProcessor(
    [
        "2mu2e_matchedLj",
    ],
    ["base_plus_gen"]
)

output = runner.run(fileset, treename="ffNtuplizer/ffNtuple", processor_instance=p)
hists = output["out"]["2Mu2E_100GeV_1p2GeV_9p6mm"]["hists"]

Output()

Output()

concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/coffea/processor/executor.py", line 1654, in _work_function
    out = processor_instance.process(events)
  File "/home/cms-jovyan/SIDM/sidm/studies/../../sidm/tools/sidm_processor.py", line 78, in process
    counts = ak.ones_like(objs[obj_name].x, dtype=np.int32)
  File "/opt/conda/lib/python3.9/site-packages/awkward/highlevel.py", line 1123, in __getattr__
    raise AttributeError(
AttributeError: no field named 'x'

(https://github.com/scikit-hep/awkward-1.0/blob/1.10.3/src/awkward/highlevel.py#L1124)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/concurrent/futures/process.py", line 246, in _process_worker
    r = call_item.fn(*call_item.args, **call_item.kwargs)
  File "/opt/conda/lib/python3.9/site-packages/coffea/processor/executor.py", line 221, in __call__


Exception: Failed processing file: WorkItem(dataset='2Mu2E_100GeV_1p2GeV_9p6mm', filename='root://xcache//store/group/lpcmetx/SIDM/ffNtupleV4/2018/SIDM_XXTo2ATo2Mu2E_mXX-100_mA-1p2_ctau-9p6_TuneCP5_13TeV-madgraph-pythia8/RunIIAutumn18DRPremix-102X_upgrade2018_realistic_v15-v1/210326_161703/0000/ffNtuple_3.root', treename='ffNtuplizer/ffNtuple', entrystart=0, entrystop=2457, fileuuid=b'D\xe0\xe2\x8c\x8eO\x11\xeb\x8ch\xeeI\x9e\x86\xbe\xef', usermeta={})

First add basic pf e, pf mu, pf photon, and dsa mu hists

In [ ]:
utilities.plot(hists["electron_n"]["2mu2e_matchedLj", :])

In [ ]:
utilities.plot(hists["electron_pt"]["2mu2e_matchedLj", :])

In [ ]:
utilities.plot(hists["electron_eta_phi"]["2mu2e_matchedLj", :, :])

In [ ]:
utilities.plot(hists["electron_lj_dR"]["2mu2e_matchedLj", :])

In [ ]:
utilities.plot(hists["electron_lj_dR_lowRange"]["2mu2e_matchedLj", :])

In [ ]:
utilities.plot(hists["photon_n"]["2mu2e_matchedLj", :])

In [ ]:
utilities.plot(hists["photon_pt"]["2mu2e_matchedLj", :])

In [ ]:
utilities.plot(hists["photon_eta_phi"]["2mu2e_matchedLj", :, :])

In [ ]:
utilities.plot(hists["photon_lj_dR"]["2mu2e_matchedLj", :])

In [ ]:
utilities.plot(hists["photon_lj_dR_lowRange"]["2mu2e_matchedLj", :])

In [ ]:
utilities.plot(hists["photon_lj_dR_reallyLowRange"]["2mu2e_matchedLj", :])

In [ ]:
utilities.plot(hists["muon_n"]["2mu2e_matchedLj", :])

In [ ]:
utilities.plot(hists["muon_pt"]["2mu2e_matchedLj", :])

In [ ]:
utilities.plot(hists["muon_eta_phi"]["2mu2e_matchedLj", :, :])

In [ ]:
utilities.plot(hists["muon_lj_dR"]["2mu2e_matchedLj", :])

In [ ]:
utilities.plot(hists["muon_lj_dR_lowRange"]["2mu2e_matchedLj", :])

In [ ]:
utilities.plot(hists["dsaMuon_n"]["2mu2e_matchedLj", :])

In [ ]:
utilities.plot(hists["dsaMuon_pt"]["2mu2e_matchedLj", :])

In [ ]:
utilities.plot(hists["dsaMuon_eta_phi"]["2mu2e_matchedLj", :, :])

In [ ]:
utilities.plot(hists["dsaMuon_lj_dR"]["2mu2e_matchedLj", :])

In [ ]:
utilities.plot(hists["dsaMuon_lj_dR_lowRange"]["2mu2e_matchedLj", :])

The photon hists in particular are interesting in that photon pT tends to ~= dark photon pT and dR(photon, LJ) tends to be exactly 0. I guess it makes sense that the majority of LJs whose constituents include photons are just a single reco photon from the dark photon decaying to two nearby electrons in ECAL. We've seen in the lj_resolution notebook that photon LJs appear to have much better pT resolution -- perhaps this is the cause.

Next, I would be interested in looking at the following:
- these same distributions after filtering the collections with pT, id, eta, and dR(LJ) criteria to only include potential LJ constituents
- which particles LJs tend to be composed of (is it more or less always 2 mu, 2 e, or 1 photon, or are more complicated combinations possible/frequent?)
- can I successfully match the reco e, mu, and photons with the gen dark photon daughters?
- do LJs sometimes miss some of the gen dark photon daughters?